In [3]:
import requests
import time

subreddit = 'stress'
maxThings = -1
printWait = 2
requestSize = 100


def requestJSON(url):
    while True:
        try:
            r = requests.get(url)
            if r.status_code != 200:
                print('error code', r.status_code)
                time.sleep(5)
                continue
            else:
                break
        except Exception as e:
            print(e)
            time.sleep(5)
            continue
    return r.json()

meta = requestJSON('https://api.pushshift.io/meta')
limitPerMinute = meta['server_ratelimit_per_minute']
requestWait = 60 / limitPerMinute

print('server_ratelimit_per_minute', limitPerMinute)

things = ('submission', 'comment')

for thing in things:
    i = 0

    with open(subreddit + '_' + thing + '_' + str(int(time.time())) + '.txt', 'w') as f:
        print('\n[starting', thing + 's]')

        if maxThings < 0:

            url = 'https://api.pushshift.io/reddit/search/'\
                  + thing + '/?subreddit='\
                  + subreddit\
                  + '&metadata=true&size=0'
            
            json = requestJSON(url)
            
            totalResults = json['metadata']['total_results']
            print('total ' + thing + 's', 'in', subreddit,':', totalResults)
        else:
            totalResults = maxThings
            print('downloading most recent', maxThings)


        created_utc = ''

        startTime = time.time()
        timePrint = startTime
        while True:
            url = 'http://api.pushshift.io/reddit/search/'\
                  + thing + '/?subreddit=' + subreddit\
                  + '&size=' + str(requestSize)\
                  + '&before=' + str(created_utc)

            json = requestJSON(url)

            if len(json['data']) == 0:
                break

            doneHere = False
            for post in json['data']:
                created_utc = post["created_utc"]
                f.write(str(post) + '\n')
                i += 1
                if i >= totalResults:
                    doneHere = True
                    break

            if doneHere:
                break
            
            if time.time() - timePrint > printWait:
                timePrint = time.time()
                percent = i / totalResults * 100
                
                timePassed = time.time() - startTime
                
                print('{:.2f}'.format(percent) + '%', '|',
                        time.strftime("%H:%M:%S", time.gmtime(timePassed)))


            time.sleep(requestWait)


server_ratelimit_per_minute 120

[starting submissions]
total submissions in stress : 6624
3.02% | 00:00:04
4.53% | 00:00:08
6.04% | 00:00:17
7.55% | 00:00:21
9.06% | 00:00:23
10.57% | 00:00:35
12.08% | 00:00:43
13.59% | 00:00:46
15.10% | 00:00:49
16.61% | 00:00:51
18.12% | 00:01:01
19.63% | 00:01:05
21.14% | 00:01:12
22.64% | 00:01:15
24.15% | 00:01:18
25.66% | 00:01:21
27.17% | 00:01:31
28.68% | 00:01:35
30.19% | 00:01:44
31.70% | 00:01:48
33.21% | 00:01:55
34.72% | 00:01:57
36.23% | 00:02:00
37.74% | 00:02:04
39.25% | 00:02:17
40.76% | 00:02:20
42.27% | 00:02:27
43.78% | 00:02:30
45.29% | 00:02:32
46.80% | 00:02:42
48.31% | 00:02:45
49.82% | 00:02:51
51.33% | 00:02:53
52.84% | 00:02:55
54.35% | 00:02:59
55.86% | 00:03:12
57.37% | 00:03:14
58.88% | 00:03:20
60.39% | 00:03:24
61.90% | 00:03:27
63.41% | 00:03:37
64.90% | 00:03:40
66.41% | 00:03:45
67.92% | 00:03:50
69.43% | 00:03:52
70.94% | 00:03:55
72.45% | 00:03:58
73.96% | 00:04:04
75.47% | 00:04:08
76.98% | 00:04:11
error code 522